In [22]:
import numpy as np
import pandas as pd
import math
import re

In [23]:
def buildArrayFromLines(lines):
    
    tmp_arr = []
    for line in lines:
        tmp_line = []    
        
        for c in line:            
            if c == ".":
                tmp_line += [0]
            elif c == "#":
                tmp_line += [1]
            elif c == "\n":
                pass
            else:
                print("THIS SHOULDN'T HAVE HAPPENED: ",print(c),"-",line,"-")
        tmp_arr.append(tmp_line)
    
    
    return np.array(tmp_arr)
    

In [24]:
def buildAllArrays(lines):
    res = []
    
    accum_lines = []
    for line in lines:
        line=line.strip()
        
        if len(line) == 0:
            arr_np = buildArrayFromLines(accum_lines)           
            res.append(arr_np)
            accum_lines = []
        else:
            accum_lines.append(line)
    
    if len(accum_lines) > 0:
        arr_np = buildArrayFromLines(accum_lines)           
        res.append(arr_np)
    return res

In [25]:
# OK
def parseFile(fileName):   
    with open(fileName) as file:
        lines =  file.readlines()    
   
    map_list = buildAllArrays(lines)
        
    return map_list


In [27]:
def findReflectionLinesScore(arr_np,bInCols=True):
    res = []
    #out = -1
    arr_fwd = arr_np.copy()
    
    if bInCols:
        arr_fwd = np.transpose(arr_fwd)
    
    # now we can always do it by row
    
    
    shape_X,_ = arr_fwd.shape

    
    
    for i in range(1,shape_X):
        # i means reflection line BEFORE i-th column
        
        my_len = min(i,shape_X-i)
        st_col = i-my_len
        #print(shape_Y,st_col,i,i+my_len)
        
        a=arr_fwd[st_col:i]
        b=arr_fwd[i:i+my_len][::-1]
                
        #print(b)
        val=np.sum(abs(b-a))
        res.append((i,bInCols,val))
        
    return res

In [28]:
def scoreArray(arr_np,tgt_diff):
    cols_scores =  findReflectionLinesScore(arr_np,True)
    cols_scores_match_tgt = [x for (x,_,val) in cols_scores if val == tgt_diff]
    
    rows_scores =  findReflectionLinesScore(arr_np,False)
    rows_scores_match_tgt = [x for (x,_,val) in rows_scores if val == tgt_diff]
    
    if cols_scores_match_tgt:
        tgt_col = cols_scores_match_tgt[0]
    else:
        tgt_col = 0
    
    if rows_scores_match_tgt:
        tgt_row = rows_scores_match_tgt[0]
    else:
        tgt_row = 0
    
    return tgt_col + 100*tgt_row

In [29]:
def scoreAllArrays(arrays_list,tgt_refl_score):
    res = 0
    for my_arr in map_list:
        tmp_res = scoreArray(my_arr,tgt_refl_score)
        #tmp_res = changedLinesScore(my_arr)
        res += tmp_res    
    return res

In [30]:
#FILE_NAME = "test.txt"
FILE_NAME = "input.txt"

In [31]:
map_list = parseFile(FILE_NAME)
resA = scoreAllArrays(map_list,0)
resB = scoreAllArrays(map_list,1)

print("Result part A:",resA)
print("Result part B:",resB)

Result part A: 31956
Result part B: 37617


# Testing things out for numpy arrays

In [32]:
x = np.zeros(10).reshape((2,5))

In [33]:
x

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [34]:
x[1,0]=2
x[0,1]=3

In [35]:
x[::-1]

array([[2., 0., 0., 0., 0.],
       [0., 3., 0., 0., 0.]])

In [36]:
x[:,::-1]

array([[0., 0., 0., 3., 0.],
       [0., 0., 0., 0., 2.]])

In [37]:
z = x.copy()
z

array([[0., 3., 0., 0., 0.],
       [2., 0., 0., 0., 0.]])

In [38]:
z[0,0]=-1

In [39]:
z

array([[-1.,  3.,  0.,  0.,  0.],
       [ 2.,  0.,  0.,  0.,  0.]])

In [40]:
x

array([[0., 3., 0., 0., 0.],
       [2., 0., 0., 0., 0.]])

In [41]:
y="#.##..##."

In [42]:
res = []
for a in y:
    if a == ".":
        res += [0]
    else:
        res += [1]
print(res)
print(np.array([res,res]))

[1, 0, 1, 1, 0, 0, 1, 1, 0]
[[1 0 1 1 0 0 1 1 0]
 [1 0 1 1 0 0 1 1 0]]


In [26]:
def findReflectionsLines(arr_np,bInCols=True):
    res = []
    arr_fwd = arr_np.copy()
    
    if not bInCols:
        arr_fwd = np.transpose(arr_fwd)
    
    # now we can do it by column
    
    
    _,shape_Y = arr_fwd.shape
    #print(shape_Y)
    #
    #print("---")   
    #print(arr_fwd)
    #print("-")
    
    
    for i in range(1,shape_Y):
        # i means reflection line BEFORE i-th column
        
        my_len = min(i,shape_Y-i)
        st_col = i-my_len
        #print(shape_Y,st_col,i,i+my_len)
        
        a=arr_fwd[:,st_col:i]
        b=arr_fwd[:,i:i+my_len][:,::-1]
                
        #print(b)
        
        if np.array_equal(a,b):
            res.append(i)
        #print(a)
        #print("-")
        #print(b)
        #print("--")
    return res, i
        
